In [1]:
import open3d as o3d
import laspy
import numpy as np
import pandas as pd
import CSF
import matplotlib.pyplot as plt
%matplotlib inline

# LASer file analysis

In [2]:
las = laspy.read("./db/pcl/tree.las")
print(len(las.points))
print(list(las.point_format.dimension_names))

pcl = o3d.geometry.PointCloud()

points = np.vstack((las.x, las.y, las.z))
colors = np.vstack((las.red, las.green, las.blue)).astype(np.float64)
colors /= 65545

pcl.points = o3d.utility.Vector3dVector(points.transpose())
pcl.colors = o3d.utility.Vector3dVector(colors.transpose())
center = pcl.get_center()
pcl.translate(-center)
o3d.visualization.draw_geometries([pcl])

51292
['X', 'Y', 'Z', 'intensity', 'return_number', 'number_of_returns', 'scan_direction_flag', 'edge_of_flight_line', 'classification', 'synthetic', 'key_point', 'withheld', 'scan_angle_rank', 'user_data', 'point_source_id', 'gps_time', 'red', 'green', 'blue']


2025-02-04 21:57:15.569 python[6926:4100627] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-04 21:57:15.569 python[6926:4100627] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [3]:
print(las.header.point_format)
print(las.header)
for i in list(las.header.vlrs):
	print(i)

<PointFormat(3, 0 bytes of extra dims)>
<LasHeader(1.2, <PointFormat(3, 0 bytes of extra dims)>)>
<GeoKeyDirectoryVlr(7 geo_keys)>
<GeoAsciiParamsVlr(['SWEREF99 TM|SWEREF99|', ''])>
<VLR(user_id: 'liblas', record_id: '2112', data len: 591)>


# LiDAR Preprocessing with CSF

In [7]:
xyz = np.vstack((las.x, las.y ,las.z)).transpose()
points = las.points
csf = CSF.CSF()
csf.params.bSloopSmooth = False
csf.params.cloth_resolution = 0.2
csf.params.interations = 500
csf.params.class_threshold = 4.0


In [13]:
csf.setPointCloud(xyz)
ground = CSF.VecInt()
non_ground = CSF.VecInt()
csf.do_filtering(ground, non_ground)
outfile = laspy.LasData(las.header)
outfile.points = points[np.array(non_ground)]

[0] Configuring terrain...
[0]  - bbMin: 508484 -262.354 6.30093e+06
[0]  - bbMax: 508491 -240.33 6.30094e+06
[0] Configuring cloth...
[0]  - width: 43 height: 41
[0] Rasterizing...
[0] Simulating...


In [14]:
tree = o3d.geometry.PointCloud()
tree_points = np.vstack((outfile.x, outfile.y, outfile.z))
tree.points = o3d.utility.Vector3dVector(tree_points.transpose())
o3d.visualization.draw_geometries([tree])

# LiDAR Segmentation with DBSCAN

In [ ]:
pcl = pcl.voxel_down_sample(voxel_size=0.5)
o3d.visualization.draw_geometries([pcl])

In [ ]:
clss = np.unique(las.classification)
print(clss)

In [ ]:
mask = las.classification == 2

xyz_t = np.vstack((las.x[mask], las.y[mask], las.z[mask]))

ground_pts = o3d.geometry.PointCloud()
ground_pts.points = o3d.utility.Vector3dVector(xyz_t.transpose())

o3d.visualization.draw_geometries([ground_pts])

In [ ]:
notground = las.classification == 1
points = np.vstack((las.x[notground], las.y[notground], las.z[notground]))
#colors = np.vstack((las.red[notground], las.green[notground], las.blue[notground]))
pcl = o3d.geometry.PointCloud()
pcl.points = o3d.utility.Vector3dVector(points.transpose())
#pclnotground.colors = o3d.utility.Vector3dVector(colors.transpose())
o3d.visualization.draw_geometries([pcl])

In [ ]:
nn_distance = np.mean(pcl.compute_nearest_neighbor_distance())
print(nn_distance)

In [ ]:
epsilon = 2
min_cluster_points = 100
labels = np.array(pcl.cluster_dbscan(eps=epsilon, min_points=min_cluster_points))
max_label = labels.max()
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 1
pcl.colors = o3d.utility.Vector3dVector(colors[:,:3])
o3d.visualization.draw_geometries([pcl])